extract text from image

In [ ]:
pip install pytesseract pillow


In [ ]:
import pytesseract
from PIL import Image


In [ ]:
image_path = "C:\\Users\\LCSS\\Desktop\\DS sip\\Capture.PNG"
image = Image.open(image_path)


In [ ]:
text = pytesseract.image_to_string(image)
print(text)


In [ ]:
import csv

text = pytesseract.image_to_string(image)
image_name = "image.jpg"
timestamp = "2023-06-24"

output_file = "output.csv"
with open(output_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Image Name", "Timestamp", "Extracted Text"])
    writer.writerow([image_name, timestamp, text])


In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("output.csv")

# Specify the column containing the text
text_column = "Text"

# Specify the column containing the font type
font_column = "Font"

# Iterate over each row and extract the font type for each text entry
for index, row in df.iterrows():
    text = row[text_column]
    font_type = row[font_column]
    print(f"Text: {text}, Font Type: {font_type}")


Font dection github classifier

In [ ]:
pip install ocrd_typegroup_classifier

In [ ]:

pip install ocrd

In [ ]:
pip install virtualenv


In [ ]:
cd ocrd_typegroups_classifier/

In [ ]:
make install

In [ ]:
sh test/test.sh

In [ ]:
pip install PyTorch

In [ ]:
pip install ocrd_typegroups_classifier


In [ ]:
from ocrd_typegroups_classifier.typegroups_classifier import TypegroupsClassifier

tgc = TypegroupsClassifier.load('ocrd_typegroups_classifier/models/densenet121.tgc')
tgc.model.save_state_dict('model.pt')


### font recognition classsifier

In [1]:
pip install keras

In [2]:
pip install tensorflow


  Using cached tensorflow_intel-2.12.0-cp311-cp311-win_amd64.whl (272.9 MB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.13.0
    Uninstalling tensorboard-2.13.0:
      Successfully uninstalled tensorboard-2.13.0
  Attempting uninstall: tensorflow-intel
    Found existing installation: tensorflow-intel 2.13.0
    Uninstalling tensorflow-intel-2.13.0:
      Successfully uninstalled tensorflow-intel-2.13.0
N

In [3]:
import tensorflow as tf


In [4]:
pip install imutils


Note: you may need to restart the kernel to use updated packages.


In [6]:
from matplotlib.pyplot import imshow
import matplotlib.cm as cm
import matplotlib.pylab as plt
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import PIL
from PIL import ImageFilter
import cv2
import itertools
import random
import keras
import imutils
from imutils import paths
import os
from keras import optimizers
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import callbacks
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D , UpSampling2D ,Conv2DTranspose
from keras import backend as K

%matplotlib inline

In [ ]:
def pil_image(img_path):
    pil_im =PIL.Image.open(img_path).convert('L')
    pil_im=pil_im.resize((105,105))
    #imshow(np.asarray(pil_im))
    return pil_im

In [ ]:
def noise_image(pil_im):
    # Adding Noise to image
    img_array = np.asarray(pil_im)
    mean = 0.0   # some constant
    std = 5   # some constant (standard deviation)
    noisy_img = img_array + np.random.normal(mean, std, img_array.shape)
    noisy_img_clipped = np.clip(noisy_img, 0, 255)
    noise_img = PIL.Image.fromarray(np.uint8(noisy_img_clipped))
    noise_img=noise_img.resize((105,105))
    return noise_img
def blur_image(pil_im):
    #Adding Blur to image 
    blur_img = pil_im.filter(ImageFilter.GaussianBlur(radius=3))
    blur_img=blur_img.resize((105,105))
    return blur_img
def affine_rotation(img):
    rows, columns = img.shape

    point1 = np.float32([[10, 10], [30, 10], [10, 30]])
    point2 = np.float32([[20, 15], [40, 10], [20, 40]])

    A = cv2.getAffineTransform(point1, point2)

    output = cv2.warpAffine(img, A, (columns, rows))
    affine_img = PIL.Image.fromarray(np.uint8(output))
    affine_img=affine_img.resize((105,105))
    return affine_img
def gradient_fill(image):
    laplacian = cv2.Laplacian(image,cv2.CV_64F)
    laplacian = cv2.resize(laplacian, (105, 105))
    return laplacian

In [ ]:
data_path = "font_patch/" #Link to all samples created
data=[]
labels=[]
imagePaths = sorted(list(paths.list_images(data_path)))
random.seed(42)
random.shuffle(imagePaths)
#These were the 5 fonts taken as sample
def conv_label(label):
    if label == 'Lato':
        return 0
    elif label == 'Raleway':
        return 1
    elif label == 'Roboto':
        return 2
    elif label == 'Sansation':
        return 3
    elif label == 'Walkway':
        return 4
augument=["blur","noise","affine","gradient"]
a=itertools.combinations(augument, 4)

for i in list(a): 
    print(list(i))

In [ ]:
counter=0
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    label = conv_label(label)
    pil_img = pil_image(imagePath)
    #imshow(pil_img)
    
    # Adding original image
    org_img = img_to_array(pil_img)
    #print(org_img.shape)
    data.append(org_img)
    labels.append(label)
    
    augument=["noise","blur","affine","gradient"]
    for l in range(0,len(augument)):
    
        a=itertools.combinations(augument, l+1)

        for i in list(a): 
            combinations=list(i)
            print(len(combinations))
            temp_img = pil_img
            for j in combinations:
            
                if j == 'noise':
                    # Adding Noise image
                    temp_img = noise_image(temp_img)
                    
                elif j == 'blur':
                    # Adding Blur image
                    temp_img = blur_image(temp_img)
                    #imshow(blur_img)
                    
    
                elif j == 'affine':
                    open_cv_affine = np.array(pil_img)
                    # Adding affine rotation image
                    temp_img = affine_rotation(open_cv_affine)

                elif j == 'gradient':
                    open_cv_gradient = np.array(pil_img)
                    # Adding gradient image
                    temp_img = gradient_fill(open_cv_gradient)
  
            temp_img = img_to_array(temp_img)
            data.append(temp_img)
            labels.append(label)

In [ ]:
data = np.asarray(data, dtype="float32") / 255.0
labels = np.array(labels, dtype="int")
print("Success")

if len(data) > 0:
    (trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)
    trainY = to_categorical(trainY, num_classes=5)
    testY = to_categorical(testY, num_classes=5)

    # Data augmentation
    batch_size = 32
    aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1,
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    augmented_data = aug.flow(trainX, trainY, batch_size=batch_size)
else:
    print("Insufficient data to perform the train-test split.")


In [ ]:
pip install pydot

In [13]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    UpSampling2D,
    Conv2DTranspose,
    BatchNormalization,
    Flatten,
    Dense,
    Dropout,
)
from tensorflow.keras.optimizers import legacy as keras_legacy_optimizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model

K.set_image_data_format('channels_last')

def create_model():
    model = Sequential()

    # Conv Layers
    model.add(Conv2D(64, kernel_size=(48, 48), activation='relu', input_shape=(105, 105, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, kernel_size=(24, 24), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2DTranspose(128, (24, 24), strides=(2, 2), activation='relu', padding='same', kernel_initializer='uniform'))
    model.add(UpSampling2D(size=(2, 2)))

    model.add(Conv2DTranspose(64, (12, 12), strides=(2, 2), activation='relu', padding='same', kernel_initializer='uniform'))
    model.add(UpSampling2D(size=(2, 2)))

    # Conv Layers
    model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))
    model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))
    model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))

    model.add(Flatten())

    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(2383, activation='relu'))
    model.add(Dense(5, activation='softmax'))

    return model

# Constants
batch_size = 128
epochs = 50

# Create model
model = create_model()

# Optimizer and Compilation
sgd = keras_legacy_optimizer.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
filepath = "C:\\Users\\LCSS\\Desktop\\DS sip\\page0.jpg"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [early_stopping, checkpoint]

# Print model summary
model.summary()

# Visualize model architecture
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

# Train the model
# model.fit(...)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 58, 58, 64)        147520    
                                                                 
 batch_normalization (BatchN  (None, 58, 58, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 29, 29, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 6, 128)         4718720   
                                                                 
 batch_normalization_1 (Batc  (None, 6, 6, 128)        512       
 hNormalization)                                                 
                                                        

c:\Users\LCSS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [14]:
model.fit(trainX, trainY,shuffle=True,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(testX, testY),callbacks=callbacks_list)
score = model.evaluate(testX, testY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: name 'trainX' is not defined

In [ ]:
from keras.models import load_model
model = load_model("C:\\Users\\LCSS\\Desktop\\DS sip\\page0.jpg")
score = model.evaluate(testX, testY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
img_path="sample/sample.jpg"
pil_im =PIL.Image.open(img_path).convert('L')
pil_im=blur_image(pil_im)
org_img = img_to_array(pil_im)
def rev_conv_label(label):
    if label == 0 :
        return 'Lato'
    elif label == 1:
        return 'Raleway'
    elif label == 2 :
        return 'Roboto'
    elif label == 3 :
        return 'Sansation'
    elif label == 4:
        return 'Walkway'
data=[]
data.append(org_img)
data = np.asarray(data, dtype="float") / 255.0
y = model.predict(data)
y = np.round(y).astype(int)
label = rev_conv_label(y[0,0])
fig, ax = plt.subplots(1)
ax.imshow(pil_im, interpolation='nearest', cmap=cm.gray)
ax.text(5, 5, label , bbox={'facecolor': 'white', 'pad': 10})
plt.show()